In [2]:
import os
import pandas as pd
import numpy as np
import librosa
from keras.models import model_from_json

In [8]:

json_file_path = r'target\project_emotion_detection_final_version2.json'
weights_file_path = r'target\project_emotion_detection_final_version2.weights.h5'

# Load model
with open(json_file_path, 'r') as json_file:
    loaded_model_json = json_file.read()

loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(weights_file_path)

res_type_s = 'kaiser_best'
duration_s = None  
sample_rate_s = 16000
sample_rate_mfcc = 16000
offset_s = 0.0
n_mfcc = 40
axis_mfcc = 1  

emotions = ['Cáu Giận', 'Lạnh Lùng', 'Mệt Mỏi', 'Thân Thiện', 'Vui Vẻ']

dataset_folder = r'C:\Users\Kin Tu\Documents\RecordProcessing\Audio\Processed\Thân Thiện_Mệt mỏi_Lạnh Lùng_Cáu giận\Processed'
output_csv = r'predicted_emotions_with_probabilities.csv'

def extract_features(file_path):
    X, sample_rate = librosa.load(file_path, 
                                  res_type=res_type_s, 
                                  duration=duration_s, 
                                  sr=sample_rate_s, 
                                  offset=offset_s)
    mfccs = librosa.feature.mfcc(y=X, sr=sample_rate_mfcc, n_mfcc=n_mfcc)
    mfccs_mean = np.mean(mfccs, axis=axis_mfcc)
    return mfccs_mean

# Dự đoán cảm xúc từ file WAV
def predict_emotion(file_path):
    try:
        features = extract_features(file_path)
        features_df = pd.DataFrame([features])
        prediction = loaded_model.predict(features_df)
        predicted_class = np.argmax(prediction, axis=1)
        predicted_probabilities = prediction[0]  # Lấy xác suất từng lớp
        return emotions[predicted_class[0]], predicted_probabilities
    except Exception as e:
        return f"Error: {e}", None

results = []

for file_name in os.listdir(dataset_folder):
    if file_name.endswith('.wav'):  # Chỉ xử lý file WAV
        file_path = os.path.join(dataset_folder, file_name)
        predicted_emotion, predicted_probabilities = predict_emotion(file_path)
        if predicted_probabilities is not None:
            results.append({
                'File Name': file_name,
                'Predicted Emotion': predicted_emotion,
                # 'Probabilities': ', '.join([f"{emotions[i]}: {predicted_probabilities[i]:.2f}" for i in range(len(emotions))])
            })
        else:
            results.append({
                'File Name': file_name,
                'Predicted Emotion': predicted_emotion,
                'Probabilities': 'Error'
            })

results_df = pd.DataFrame(results)
results_df.to_csv(output_csv, index=False, encoding='utf-8')

print(f"Predicted emotions and probabilities saved to {output_csv}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Predicted emotions and probabilities saved to predicted_emotions_with_probabilities.csv
